In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/latest_data/PA_DOC_testing_data.csv')

In [40]:
most_recent = df.groupby('SCI').last()

In [41]:
most_recent['inmate_tests'] = most_recent['inmate_positive'] + most_recent['inmate_negative'] + most_recent['inmate_pending']
most_recent['inmate_current_cases'] = most_recent['inmate_positive'] - most_recent['inmate_recovered']
most_recent['inmate_current_pr'] = most_recent['inmate_positive_D'] / (most_recent['inmate_negative_D'] + most_recent['inmate_positive_D'])

In [43]:
most_recent = most_recent.reset_index()
most_recent = most_recent[['inmate_tests','inmate_current_cases','inmate_death','SCI']]

In [44]:
most_recent = most_recent.fillna(0)

In [47]:
most_recent[['inmate_tests','inmate_current_cases','inmate_death']]=most_recent[['inmate_tests','inmate_current_cases','inmate_death']].astype(int)

In [49]:

most_recent.to_csv('../_data/most_recent_inmateData.csv')